In [30]:
import pandas as pd
import os
import datetime
import re

### convert datetime to two separate columns date and time 

In [31]:
def feature_eng(df, column):
    df['pickup_date'] = [d.date() for d in pd.to_datetime(df[column])]
    df.drop(column, axis=1, inplace=True)
    df['count'] = 1
    return df

In [32]:
def get_files_list(path):
    return os.listdir(path)

In [50]:
def update_list_files():
    list_files = get_files_list('data/')
    list_output = get_files_list('new_data/')
    list_output = [item.replace('.csv', '') for item in list_output]
    for file in list_output:
        #print(file)
        if file in list_files:
            list_files.remove(file)
            #print(len(list_files))
    return list_files

In [51]:
a = update_list_files()


In [52]:
a

[]

### create light files

In [53]:
list_files = update_list_files()
for file_name in list_files:
    print('reading {} file'.format(file_name))
    import pyarrow.parquet as pq
    pfile = pq.read_table(file_name)
    #print("Column names: {}".format(pfile.column_names))
    column = [item for item in pfile.column_names if 'pickup_datetime' in item.lower()][0]
    f =  pd.read_parquet('data/' + file_name, engine='pyarrow', columns = [column])
    
    f = feature_eng(f, column)
    f = f.groupby('pickup_date').count()
    f.to_csv('new_data/' + file_name + '.csv')        

### load and concatenate files

In [54]:
full_df = pd.DataFrame()
list_files = get_files_list('new_data/')
for file_name in list_files:
    #print('reading {} file'.format(file_name))
    f =  pd.read_csv('new_data/' + file_name)
    taxi_type = re.split('_', file_name)[0]
    f['type'] = taxi_type
    full_df = full_df.append(f)

In [55]:
full_df.head()

pickup_date   count   type
0  2019-02-01  860161  fhvhv
1  2019-02-02  866393  fhvhv
2  2019-02-03  756721  fhvhv
3  2019-02-04  592170  fhvhv
4  2019-02-05  576476  fhvhv

In [56]:
final_dataset = full_df.groupby('pickup_date').sum()

In [57]:
final_dataset

count
pickup_date       
2001-01-01      11
2001-01-05       1
2001-01-06       2
2001-02-02       1
2002-02-02      11
...            ...
2081-06-24       1
2084-11-04       8
2088-01-24       2
2090-12-31       1
2098-09-11       1

[4804 rows x 1 columns]